# Course Pattern Extraction and Application

This notebook implements a two-phase approach for:
1. Extracting graph-based patterns from a course about mathematical induction
2. Using those patterns to construct knowledge graphs for specific proof examples

---

## Phase 1: Course Pattern Extraction

First, we'll extract the graph-based pattern from the mathematical induction course.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os

# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [2]:
from IPython.display import display, Latex
import re
from src.utils.file_utils import read_proof
from src.phase1.extract_triplets import extract_triplets
from src.utils.neo4j_utils import Neo4JUtils

# Load the course 1 content
course_latex_1 = read_proof("../../data/courses/mixed/prob1_course.tex")

# Load the course 2 content
course_latex_2 = read_proof("../../data/courses/mixed/prob2_course.tex")

# Load the course 3 content
course_latex_3 = read_proof("../../data/courses/mixed/prob3_course.tex")

### Extract Course Pattern

We'll use a specialized prompt to extract the pattern of mathematical induction from the course content.

In [3]:
SYSTEM_PROMPT = """You are an expert in mathematical proof/reasoning analysis, specializing in extracting structured knowledge graph from mathematical texts. Your task is to identify key detailed steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning."""

COURSE_PATTERN_PROMPT = """
Given the following mathematical course content in LaTeX format, extract a VERY DETAILED step-by-step explanatory chain that represents the calculation process. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish.

Focus on identifying:
1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)
2. The precise mathematical operations performed at each step
3. The exact sequence of operations, with clear predecessor-successor relationships
4. Intermediate results at each calculation stage
5. The mathematical justification for each step (e.g., "By the associative property", "By substituting value from step 3")

IMPORTANT: If you see examples in the course content, do not extract them as separate graphs. The examples are only included to help you understand the calculation process better. Focus only on extracting the general calculation pattern/process.

The final knowledge graph MUST:
1. Have clearly marked START node(s) (the initial problem statement)
2. Have clearly marked END node(s) (the final result)
3. Include ALL intermediate calculation steps with no gaps in reasoning
4. Form a single connected component with a clear directional flow
5. Use relationship types that precisely describe the mathematical operation performed (e.g., "applies_distributive_property", "substitutes_value", "simplifies_expression")

Extract triplets in the given form of structured output that represent this detailed calculation process.

Course Content:
```
{proof}
```
"""


# Extract the course pattern
course_pattern_1 = extract_triplets(
    proof=course_latex_1,
    custom_prompt=COURSE_PATTERN_PROMPT,
    system_message=SYSTEM_PROMPT,
)
print(course_pattern_1)

# Store the pattern in Neo4j
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))
neo4j.clean_database()
neo4j.store_triplets(course_pattern_1, "course_pattern_1")

# Create a prompt for the second course using the first pattern
COURSE_PATTERN_TRANSFER_PROMPT = """
Given the following mathematical course content in LaTeX format and a previously extracted calculation pattern, extract a VERY DETAILED step-by-step explanatory chain for this new content.

The previously extracted pattern is:
```
{previous_pattern}
```

IMPORTANT: The new course (multiplication) builds upon concepts from the previous course (addition). Your extracted knowledge graph MUST maintain this hierarchical relationship by:
1. Incorporating the relevant nodes and relationships from the previous pattern when they are used in the new content
2. Adding new nodes and relationships specific to the new calculation process
3. Creating explicit connections between the previous concepts and the new ones
4. Preserving the hierarchical structure where multiplication operations may use addition operations as sub-steps

Use this pattern as a guide, but adapt it to the specific calculation process in the new content. You can reuse elements from the previous pattern if they apply to the new content. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish in this new content.

Focus on identifying:
1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)
2. The precise mathematical operations performed at each step
3. The exact sequence of operations, with clear predecessor-successor relationships
4. Intermediate results at each calculation stage
5. The mathematical justification for each step (e.g., "By the associative property", "By substituting value from step 3")

IMPORTANT: If you see examples in the course content, do not extract them as separate graphs. The examples are only included to help you understand the calculation process better. Focus only on extracting the general calculation pattern/process.

The final knowledge graph MUST:
1. Have clearly marked START node(s) (the initial problem statement)
2. Have clearly marked END node(s) (the final result)
3. Include ALL intermediate calculation steps with no gaps in reasoning
4. Form a single connected component with a clear directional flow
5. Use relationship types that precisely describe the mathematical operation performed (e.g., "applies_distributive_property", "substitutes_value", "simplifies_expression")
6. Show how multiplication operations may use addition operations as sub-components

Extract triplets in the given form of structured output that represent this detailed calculation process.

New Course Content:
```
{{proof}}
```
"""

# Extract the pattern for the second course using the first pattern as a guide
course_pattern_2 = extract_triplets(
    proof=course_latex_2,
    custom_prompt=COURSE_PATTERN_TRANSFER_PROMPT.format(
        previous_pattern=course_pattern_1
    ),
    system_message=SYSTEM_PROMPT,
)
print(course_pattern_2)

# Store the second pattern in Neo4j
neo4j.store_triplets(course_pattern_2, "course_pattern_2")

# Create a prompt for the third course using both previous patterns
COURSE_PATTERN_TRANSFER_PROMPT_3 = """
Given the following mathematical course content in LaTeX format and two previously extracted calculation patterns, extract a VERY DETAILED step-by-step explanatory chain for this new content.

The previously extracted patterns are:

Pattern 1 (Addition):
```
{pattern_1}
```

Pattern 2 (Multiplication):
```
{pattern_2}
```

IMPORTANT: The new course (distributive property) builds upon concepts from both previous courses (addition and multiplication). Your extracted knowledge graph MUST maintain this hierarchical relationship by:
1. Incorporating the relevant nodes and relationships from both previous patterns when they are used in the new content
2. Adding new nodes and relationships specific to the new calculation process
3. Creating explicit connections between the previous concepts and the new ones
4. Preserving the hierarchical structure where distributive property operations may use both multiplication and addition operations as sub-steps

Use these patterns as a guide, but adapt them to the specific calculation process in the new content. You can reuse elements from the previous patterns if they apply to the new content. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish in this new content.

Focus on identifying:
1. Every individual calculation step, no matter how small (e.g., "Apply distributive property", "Multiply terms", etc.)
2. The precise mathematical operations performed at each step
3. The exact sequence of operations, with clear predecessor-successor relationships
4. Intermediate results at each calculation stage
5. The mathematical justification for each step (e.g., "By the distributive property", "By substituting value from step 3")

IMPORTANT: If you see examples in the course content, do not extract them as separate graphs. The examples are only included to help you understand the calculation process better. Focus only on extracting the general calculation pattern/process.

The final knowledge graph MUST:
1. Have clearly marked START node(s) (the initial problem statement)
2. Have clearly marked END node(s) (the final result)
3. Include ALL intermediate calculation steps with no gaps in reasoning
4. Form a single connected component with a clear directional flow
5. Use relationship types that precisely describe the mathematical operation performed (e.g., "applies_distributive_property", "multiplies_terms", "simplifies_expression")
6. Show how distributive property operations may use both multiplication and addition operations as sub-components

Extract triplets in the given form of structured output that represent this detailed calculation process.

New Course Content:
```
{{proof}}
```
"""

# Extract the pattern for the third course using the first and second patterns as guides
course_pattern_3 = extract_triplets(
    proof=course_latex_3,
    custom_prompt=COURSE_PATTERN_TRANSFER_PROMPT_3.format(
        pattern_1=course_pattern_1, pattern_2=course_pattern_2
    ),
    system_message=SYSTEM_PROMPT,
)
print(course_pattern_3)

# Store the third pattern in Neo4j
neo4j.store_triplets(course_pattern_3, "course_pattern_3")

[SystemMessage(content='You are an expert in mathematical proof/reasoning analysis, specializing in extracting structured knowledge graph from mathematical texts. Your task is to identify key detailed steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nGiven the following mathematical course content in LaTeX format, extract a VERY DETAILED step-by-step explanatory chain that represents the calculation process. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish.\n\nFocus on identifying:\n1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)\n2. The precise mathematical operations performed at each step\n3. The exact sequence of operations, with clear predecessor-successor relationships\n4. Intermediate r

## Phase 2: Create a Pattern from another Pattern

Now we'll use the extracted pattern 1 to construct knowledge graph for new pattern 2.

In [5]:
# Load the course 1 content
course_latex_1 = read_proof("../../data/courses/addition/course.tex")

# Load the course 2 content
course_latex_2 = read_proof("../../data/courses/multiplication/course.tex")

In [ ]:
SYSTEM_PROMPT = """You are an expert in mathematical proof/reasoning analysis, specializing in extracting structured knowledge graph from mathematical texts. Your task is to identify key detailed steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning."""

COURSE_PATTERN_PROMPT = """
Given the following mathematical course content in LaTeX format, extract a VERY DETAILED step-by-step explanatory chain that represents the calculation process. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish.

Focus on identifying:
1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)
2. The precise mathematical operations performed at each step
3. The exact sequence of operations, with clear predecessor-successor relationships
4. Intermediate results at each calculation stage
5. The mathematical justification for each step (e.g., "By the associative property", "By substituting value from step 3")

IMPORTANT: If you see examples in the course content, do not extract them as separate graphs. The examples are only included to help you understand the calculation process better. Focus only on extracting the general calculation pattern/process.

The final knowledge graph MUST:
1. Have clearly marked START node(s) (the initial problem statement)
2. Have clearly marked END node(s) (the final result)
3. Include ALL intermediate calculation steps with no gaps in reasoning
4. Form a single connected component with a clear directional flow
5. Use relationship types that precisely describe the mathematical operation performed (e.g., "applies_distributive_property", "substitutes_value", "simplifies_expression")

Extract triplets in the given form of structured output that represent this detailed calculation process.

Course Content:
```
{proof}
```
"""


# Extract the course pattern
course_pattern = extract_triplets(
    proof=course_latex_1,
    custom_prompt=COURSE_PATTERN_PROMPT,
    system_message=SYSTEM_PROMPT,
)
print(course_pattern)

# Store the pattern in Neo4j
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))
neo4j.clean_database()
neo4j.store_triplets(course_pattern, "course_1_pattern")

In [14]:
PROOF_PATTERN_APPLICATION_PROMPT = """
Given the following mathematical proof and the given pattern of mathematical proof extracted from the course, construct a knowledge graph that follows the given pattern.

The pattern components are:
{course_pattern}

For the given proof, extract triplets in the form <Source Entity, Relationship, Target Entity> that:
1. Follow the structure of mathematical proof pattern
2. Map to the steps identified in the course pattern
3. Capture the specific details and relationships in this proof which may be different from the course pattern

Proof:
{{proof}}
"""

# # Format the prompt with course_pattern
# formatted_prompt = PROOF_PATTERN_APPLICATION_PROMPT.format(
#     course_pattern=course_pattern
# )

# # Pass the formatted prompt and proof_content to extract_triplets
# proof_triplets = extract_triplets(proof, formatted_prompt)

# # # Apply the pattern to the proof
# # proof_triplets = extract_triplets(proof_content, PROOF_PATTERN_APPLICATION_PROMPT)

# # Store the proof graph in Neo4j
# neo4j.store_triplets(proof_triplets, "proof_example")

# Format the prompt with course_pattern
formatted_prompt = PROOF_PATTERN_APPLICATION_PROMPT.format(
    course_pattern=course_pattern
)

# Pass the formatted prompt and proof_content to extract_triplets
proof_triplets = extract_triplets(proof, formatted_prompt)

# Clean the database to remove any existing graphs
neo4j.clean_database()

# Store the course pattern graph
neo4j.store_triplets(course_pattern, "course_pattern")

# Store the proof graph as a separate graph
neo4j.store_triplets(proof_triplets, "proof_example")


# Extract the course pattern
course_pattern = extract_triplets(
    proof, custom_prompt=COURSE_PATTERN_PROMPT, system_message=SYSTEM_PROMPT
)
print(course_pattern)

# Display visualization queries
print(neo4j.get_visualization_queries())

[SystemMessage(content='You are a helpful assistant that extracts entities and relations from mathematical proofs.', additional_kwargs={}, response_metadata={}), HumanMessage(content="\nGiven the following mathematical proof and the given pattern of mathematical proof extracted from the course, construct a knowledge graph that follows the given pattern.\n\nThe pattern components are:\nentities=[Entity(id='1', name='Base Case', label='Base Case of Addition', type='axiom', start=True, end=False), Entity(id='2', name='Recursive Case', label='Recursive Case of Addition', type='axiom', start=False, end=False), Entity(id='3', name='Addition Definition', label='Definition of Addition', type='definition', start=False, end=True)] relations=[Relation(source='1', target='3', type='grounds', name='Base Case grounds the definition of addition'), Relation(source='2', target='3', type='grounds', name='Recursive Case grounds the definition of addition'), Relation(source='1', target='2', type='explains

## Visualization and Analysis

You can visualize and analyze the graphs in Neo4j Browser using these queries:

### View Course Pattern:
```cypher
MATCH p=()-[r]->() WHERE r.graph_type = 'course_pattern' RETURN p
```

### View Proof Graph:
```cypher
MATCH p=()-[r]->() WHERE r.graph_type = 'proof_example' RETURN p
```

### Compare Pattern and Proof:
```cypher
MATCH p=()-[r]->() RETURN p
```